## This notebook will look at all the predictions and seek to find out how many predictions at each recall interval along with calculating AUPR etc for the only has ans

In [89]:
import os

Find the correct document. As the step might have shifted between logging and saving

In [90]:
x =os.listdir('NotebookData/preds_server/experiment_data_transf/predictions')

In [91]:
filtered = [k for k in x if "roberta_base_untrained_seed_diff" in k and "epoch13" in k]
filtered


['dataset-name_CUAD_model-type_roberta_only-first-ans_True_doc-stride_256_dataset-type_eval_predict-file-version_test_model-name_roberta_base_untrained_seed_diff_global-step_6705_epoch13_predictions.json',
 'dataset-name_CUAD_model-type_roberta_only-first-ans_True_doc-stride_256_dataset-type_eval_predict-file-version_test_model-name_roberta_base_untrained_seed_diff_global-step_6466_epoch13_nbest_predictions.json',
 'dataset-name_CUAD_model-type_roberta_only-first-ans_True_doc-stride_256_dataset-type_eval_predict-file-version_test_model-name_roberta_base_untrained_seed_diff_global-step_6705_epoch13_nbest_predictions.json',
 'dataset-name_CUAD_model-type_roberta_only-first-ans_True_doc-stride_256_dataset-type_eval_predict-file-version_test_model-name_roberta_base_untrained_seed_diff_global-step_6466_epoch13_null_odds.json',
 'dataset-name_CUAD_model-type_roberta_only-first-ans_True_doc-stride_256_dataset-type_eval_predict-file-version_test_model-name_roberta_base_untrained_seed_diff_glob

Load predictions and examples

In [92]:

import json
preds_path="dataset-name_CUAD_model-type_roberta_only-first-ans_True_doc-stride_256_dataset-type_eval_predict-file-version_test_model-name_roberta_base_untrained_seed_diff_global-step_6466_epoch13_predictions.json"
preds_nbest_path = "dataset-name_CUAD_model-type_roberta_only-first-ans_True_doc-stride_256_dataset-type_eval_predict-file-version_test_model-name_roberta_base_untrained_seed_diff_global-step_6466_epoch13_nbest_predictions.json"

# Ensure correct path here
with open('NotebookData/preds_server/experiment_data_transf/predictions/' + preds_path, 'r') as f:
    preds = json.load(f)

with open('NotebookData/preds_server/experiment_data_transf/predictions/' + preds_nbest_path, 'r') as f:
    preds_nbest = json.load(f)

with open ('../data/test_full.json', 'r') as f:
    gt_dict = json.load(f)

Split the predictions and calculate AUPR

In [93]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../cuad_training/cuad_lightning/')

In [94]:
from evaluate import get_answers, load_json, get_prec_at_recall, get_precisions_recalls, get_aupr,get_jaccard, IOU_THRESH

In [95]:
def compute_precision_recall(gt_dict, preds_dict, category=None):
    tp, fp, fn = 0, 0, 0

    for key in gt_dict:
        if category and category not in key:
            continue

        substr_ok = "Parties" in key

        answers = gt_dict[key]
        preds = preds_dict[key]

        # first check if answers is empty
        if len(answers) == 0:
            if len(preds) > 0:
                fp += len(preds)  # false positive for each one

            ######################
            ### This is the issue#
            ######################
            else:
                fn += 1  # no answer and no prediction
        else:
            for ans in answers:
                assert len(ans) > 0
                # check if there is a match
                match_found = False
                for pred in preds:
                    if substr_ok:
                        is_match = get_jaccard(
                            ans, pred) >= IOU_THRESH or ans in pred
                    else:
                        is_match = get_jaccard(ans, pred) >= IOU_THRESH
                    if is_match:
                        match_found = True

                if match_found:
                    tp += 1
                else:
                    fn += 1

            # now also get any fps by looping through preds
            for pred in preds:
                # Check if there's a match. if so, don't count (don't want to double count based on the above)
                # but if there's no match, then this is a false positive.
                # (Note: we get the true positives in the above loop instead of this loop so that we don't double count
                # multiple predictions that are matched with the same answer.)
                match_found = False
                for ans in answers:
                    assert len(ans) > 0
                    if substr_ok:
                        is_match = get_jaccard(
                            ans, pred) >= IOU_THRESH or ans in pred
                    else:
                        is_match = get_jaccard(ans, pred) >= IOU_THRESH
                    if is_match:
                        match_found = True

                if not match_found:
                    fp += 1
    # print("tp: {}, fp: {}, fn: {}".format(tp, fp, fn))
    precision = tp / (tp + fp) if tp + fp > 0 else np.nan
    recall = tp / (tp + fn) if tp + fn > 0 else np.nan

    return precision, recall

In [96]:
from evaluate import get_preds
import numpy as np

def get_precisions_recalls(pred_dict, gt_dict, category=None):
    precisions = [1]
    recalls = [0]
    confs = []
    for conf in list(np.arange(0.99, 0, -0.01)) + [0.001, 0]:
        conf_thresh_pred_dict = get_preds(pred_dict, conf)
        #print("conf: {}".format(conf))
        #print("mean number of pred in questions", np.mean([len(x) for x in list(conf_thresh_pred_dict.values())]))
        #print("Sum of number of pred in questions", np.sum([len(x) for x in list(conf_thresh_pred_dict.values())]))
        #print(len(gt_dict))
        pred_d = {}
        for ele in conf_thresh_pred_dict.values():
            for idx, pred in enumerate(ele):
                if idx not in pred_d:
                    pred_d[idx] = 1
                else:
                    pred_d[idx] += 1
        prec, recall = compute_precision_recall(
            gt_dict, conf_thresh_pred_dict, category=category)
        precisions.append(prec)
        recalls.append(recall)
        confs.append(conf)
    return precisions, recalls, confs

In [97]:
len(gt_dict)

2

#Python code
precisions, recalls, confs = get_precisions_recalls(pred_dict, gt_dict)
prec_at_90_recall, _ = get_prec_at_recall(
    precisions, recalls, confs, recall_thresh=0.9)
prec_at_80_recall, _ = get_prec_at_recall(
    precisions, recalls, confs, recall_thresh=0.8)
aupr = get_aupr(precisions, recalls)

In [99]:
from evaluate import get_results,get_answers
class FakeArgs:
    def __init__(self):
        self.n_best_size = 20
        self.max_answer_length = 30
        self.do_lower_case = True
        self.verbose = True
        self.null_score_diff_threshold = 0.0
        self.version_2_with_negative = True
        self.test_examples_workers = 4
        self.test_examples_chunk_size = 100
        self.model_name="fes"
        self.model_version="fes"

def get_results(args, n_best_predictions_path, gt_dict, gt_dict_extract_answers=True, include_model_info=True, filter=None):
    """

    Args:
        args (_type_): _description_
        n_best_predictions_path (_type_): _description_
        gt_dict (_type_): _description_
        gt_dict_extract_answers (bool, optional): _description_. Defaults to True.
        include_model_info (bool, optional): _description_. Defaults to True.
        filter (_type_, optional): Filters the results to keep a specific subset of questions. Defaults to None.

    Returns:
        _type_: _description_
    """

    gt_dict = get_answers(gt_dict) if gt_dict_extract_answers else gt_dict

    pred_dict = load_json(n_best_predictions_path)

    assert sorted(list(pred_dict.keys())) == sorted(list(gt_dict.keys()))

    # Remove examples with empty answers
    to_rm = []
    if filter == "No Ans":
        to_rm = [k for k, v in gt_dict.items() if not v]
        print("Removing {} examples with empty answers".format(len(to_rm)))
    elif filter == "Has Ans":
        to_rm = [k for k, v in gt_dict.items() if v]
        print("Removing {} examples with answers".format(len(to_rm)))
    elif filter:
        to_rm = [k for k, v in gt_dict.items() if filter not in k]
        print("Removing {} examples with answers".format(len(to_rm)))
    for k in to_rm:
        gt_dict.pop(k, None)
        pred_dict.pop(k, None)

    assert sorted(list(pred_dict.keys())) == sorted(list(gt_dict.keys()))

    precisions, recalls, confs = get_precisions_recalls(pred_dict, gt_dict)
    prec_at_90_recall, _ = get_prec_at_recall(
        precisions, recalls, confs, recall_thresh=0.9)
    prec_at_80_recall, _ = get_prec_at_recall(
        precisions, recalls, confs, recall_thresh=0.8)
    aupr = get_aupr(precisions, recalls)

    # now save results as a dataframe and return
    results = {"name": args.model_name, "version": args.model_version, "aupr": aupr, "prec_at_80_recall": prec_at_80_recall,
               "prec_at_90_recall": prec_at_90_recall}
    if not include_model_info:
        del results["name"]
        del results["version"]
    return results, precisions, recalls, confs,pred_dict, gt_dict

In [100]:
res = get_results(FakeArgs(), './NotebookData/preds_server/experiment_data_transf/predictions/' + preds_nbest_path,
                    gt_dict=gt_dict, include_model_info=False, filter="Uncapped Liability")

Removing 4080 examples with answers


In [102]:
res[0]

{'aupr': 0.0699347329555811, 'prec_at_80_recall': 0, 'prec_at_90_recall': 0}

## Let's plot the rank of predictions as a scatterplot

In [81]:
import pandas as pd
data = []

for k,v in preds_nbest.items():
    if not "Uncapped Liability" in k:
        continue
    ans = gt_dict[k]
    for idx, pred in enumerate(v):
        if len(ans)==0:
            if pred['token_doc_start']==-1 and pred['token_doc_end']==-1:
                data.append([idx, pred["probability"], "Empty True", True])
            else:
                data.append([idx, pred["probability"], "Empty False", False])
        else:
            is_match=False
            for an in ans:
                if (get_jaccard(an, pred['text']) >= IOU_THRESH):
                    is_match=True
            data.append([idx, pred["probability"], f"Text {is_match}", is_match])

df = pd.DataFrame(data, columns=["idx", "prob", "category", "is_match"])

In [82]:
import plotly.express as px

px.scatter(df, x="idx", y="prob", title="Probability of Uncapped Liability", color="category")

# Why is there a bad mix between the loss and AUPR

In [104]:
#import torch nn
import torch
import torch.nn as nn


Based on the data from an actual SQuAD example, the output could looks something like this

In [105]:
# Tensor of 20 numbers uniformly distributed in the range [-9, 0]
start_logits = torch.rand(20, dtype=torch.float) * -9
start_logits[7]=5.9
# We will set the correct labels as position 7
start_positions = torch.tensor(7)

In [106]:
# Make the same for endlogits
end_logits = torch.zeros(20, dtype=torch.float) * -9
end_logits[7] = 6.9

end_positions = torch.tensor(7)


Now we have our prediction setup and can calculate the loss

In [107]:
loss_fct = nn.CrossEntropyLoss()
start_loss = loss_fct(start_logits, start_positions)
end_loss = loss_fct(end_logits, end_positions)
total_loss = (start_loss + end_loss) / 2

In [108]:
print("Start loss:", start_loss)
print("End loss:", end_loss)
print("Total loss:", total_loss)

Start loss: tensor(0.0103)
End loss: tensor(0.0190)
Total loss: tensor(0.0146)


Basically this is a very low loss as most predictions are very low in the start and end span. If we make and extra logit have a semi good output, then lets see how it affects it

In [109]:
end_logits[8]=2.9

loss_fct = nn.CrossEntropyLoss()
start_loss = loss_fct(start_logits, start_positions)
end_loss = loss_fct(end_logits, end_positions)
total_loss = (start_loss + end_loss) / 2

print("Start loss:", start_loss)
print("End loss:", end_loss)
print("Total loss:", total_loss)

Start loss: tensor(0.0103)
End loss: tensor(0.0358)
Total loss: tensor(0.0230)


The loss lose to dubles, but it's still quite low, relativly to normal predictions, but out model predictions works in terms of the AUPR. This extra logit with a slight activation will affect the total probabilities of predictions a lot, as our prediction score that we do the softmax of is the sum of start logit and end logits

In [110]:
# start logits max =5.3
# end logits max =4.2

preds = torch.tensor([9.5,5.9,6.1,6.2,6.8,5.9,6.1,6.2,6.8,5.9,6.1,6.2,6.8,5.9,6.1,6.2,6.8])
# softmax preds
torch.softmax(preds, dim=0)


tensor([0.6027, 0.0165, 0.0201, 0.0222, 0.0405, 0.0165, 0.0201, 0.0222, 0.0405,
        0.0165, 0.0201, 0.0222, 0.0405, 0.0165, 0.0201, 0.0222, 0.0405])

This means that the FP should only be added in the last couple of confidence steps

Maybe also affects the precision because we add a lot of mass outside the

In general it seems as using the sum of scores as the prediction score just decreases the confidence in top prediction scores because the mass has to be distributed so much more

### What is up with that AUPR

* The span of prediction confidence probabilities varies som damm much. It's filtered by confidence not rank
* When we start getting the other correct predictions we are all over the place